In [28]:
import numpy as np
import csv
import cv2
import os, glob
import random
import sklearn
from sklearn.model_selection import train_test_split
import time
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from matplotlib import pyplot as plt
from subprocess import check_output

In [29]:
def load_data(data_dir):
	
	# Get all subdirectories. A folder for each class
    #if os.path.isdir(os.path.join(data_dir, d))]
	directories = [d for d in os.listdir(data_dir)
					if os.path.isdir(os.path.join(data_dir, d))]
	#print(directories)
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
	labels = []
	images = []

	category = 0
	for d in directories:
		label_dir = os.path.join(data_dir, d)
		file_names = [os.path.join(label_dir, f)
					for f in os.listdir(label_dir)
					if f.endswith(".jpg")]
        # adding an early stop for sake of speed
		#stop = 0
		for f in file_names:
			img = cv2.imread(f)
			imresize = cv2.resize(img, (100, 100))
			#plt.imshow(imresize)
			images.append(imresize)
			labels.append(category)
			# remove this to use full data set
			#if stop > 30:
			#break
			#stop += 1
			#print(stop)
			# end early stop
            
		category += 1

	return images, labels

In [30]:
PATH = os.getcwd()

data_dir = PATH+'/train'
training_images, training_labels = load_data(data_dir)

# confirm that we have the data
print(len(training_images))
print(len(training_labels))
print("Training set dims")
#cv2.imshow(str(training_labels[random.randint(0,(len(training_labels)-1))]), training_images[random.randint(0,(len(training_images)-1))])
#cv2.waitKey(0)
#cv2.destroyAllWindows()

data_dir = PATH+'/validation'
validation_images, validation_labels = load_data(data_dir)


9706
9706
Training set dims


In [31]:
print("Validation set dims")
print(len(validation_images))
print(len(validation_labels))

Validation set dims
8106
8106


In [32]:
data_dir = PATH+'/test'
testing_images, testing_labels = load_data(data_dir)
print("Test set dims")
print(len(testing_images))
print(len(testing_labels))


Test set dims
3245
3245


In [33]:
X_train = np.array(training_images).astype('float32')
X_cv = np.array(validation_images).astype('float32')
X_test = np.array(testing_images).astype('float32')
X_train = X_train / 255.0
X_cv = X_cv / 255.0
X_test = X_test / 255.0

In [34]:
y_train = np.array(training_labels)
y_cv = np.array(validation_labels)
y_train = np_utils.to_categorical(y_train)
y_cv = np_utils.to_categorical(y_cv)

In [35]:
num_classes = y_cv.shape[1]
print("X_train")
print(len(X_train))
print("X_cv")
print(len(X_cv))
print("y_train")
print(len(y_train))
print("y_cv")
print(len(y_cv))
print("Number of classes")
print(num_classes)
print("Data normalized and hot encoded.")


X_train
9706
X_cv
8106
y_train
9706
y_cv
8106
Number of classes
20
Data normalized and hot encoded.


In [36]:
def create_cnn_model(num_classes, lrate):
	# Create model
	model = Sequential()
	model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(100, 100, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
	model.add (Dropout(0.2))
	model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Flatten())
	model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
	model.add(Dropout(0.5))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile Model
	epochs = 1
	decay = lrate/epochs
	sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
	model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
	print(model.summary())
	return model, epochs

In [37]:
model, epochs = create_cnn_model(num_classes, 0.01)
print(" CNN model created. ")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 100, 100, 32)      896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 80000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               40960512  
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)              

In [39]:
seed = 7
np.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=epochs, batch_size=64)


Train on 9706 samples, validate on 8106 samples
Epoch 1/1
9706/9706 [==============================] - 831s 86ms/step - loss: 0.3495 - accuracy: 0.8711 - val_loss: 0.0839 - val_accuracy: 0.9774


In [40]:
scores = model.evaluate(X_cv, y_cv, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Training Done")

8106/8106 [==============================] - 104s 13ms/step
Accuracy: 97.74%
Training Done


In [ ]:
#X_test=np_utils.to_categorical(X_test)
print("Evaluating test images ")
cnn_prediction = model.predict(X_test)
print(cnn_prediction)
print("Predictions Done")


Evaluating test images 
[[9.8226005e-01 1.0317956e-08 2.9415602e-03 ... 4.0825611e-07
  1.2168939e-06 4.1130575e-04]
 [9.9486792e-01 4.4097226e-10 8.3849370e-04 ... 2.1609715e-08
  3.0671609e-07 4.5451947e-05]
 [9.9969900e-01 1.1781318e-10 6.6733657e-05 ... 3.8660950e-09
  8.7588027e-08 2.2462367e-05]
 ...
 [2.3395035e-02 7.1150776e-06 8.9385826e-04 ... 3.8425945e-02
  1.4743971e-04 6.4761442e-01]
 [1.8847538e-02 6.2206805e-06 1.0721320e-03 ... 4.2419983e-03
  2.0557694e-04 9.1555154e-01]
 [4.3993536e-03 3.8654189e-08 3.0983926e-04 ... 4.3038838e-04
  9.4682724e-07 9.9341136e-01]]
Predictions Done
